## Set system path

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
sys.path.append(str(project_root))

import Python_scripts.Extract_db_columns.normalize_bodypart_by_id
importlib.reload(Python_scripts.Extract_db_columns.normalize_bodypart_by_id)
from Python_scripts.Extract_db_columns.normalize_bodypart_by_id import normalize_bodypart_by_id


## Connect to database

In [ ]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.30"
conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")
cursor = conn.cursor()


## Check data distribution

In [ ]:
from Python_scripts.Utility_functions.plot_column_distribution_by_id import plot_column_distribution_by_id

plot_column_distribution_by_id(conn, id=56, column_name='corner4_x')
plot_column_distribution_by_id(conn, id=56, column_name='corner4_y')
# plot_column_distribution_by_id(conn, id=56, column_name='head_x')


## Count the number of low likelihood points

In [ ]:
import numpy as np

query = "SELECT head_y FROM dlc_table WHERE id = 56;"
df = pd.read_sql_query(query, conn)

# Convert to array (handle stringified list if needed)
val = df.iloc[0]['head_y']
head_x = np.array(val if isinstance(val, list) else eval(val))

# Count NaNs
num_nans = np.isnan(head_x).sum()
print(f"NaNs in head_y: {num_nans} out of {len(head_x)}")


## Check if the function working

In [ ]:
x_norm, y_norm = normalize_bodypart_by_id(conn, id=56, bodypart='head')

if x_norm is not None:
    print("Normalized coordinates (first 5 frames):")
    print("x:", x_norm[:5])
    print("y:", y_norm[:5])

## Insert the normalized csv files into database

In [ ]:
from Python_scripts.Insert_to_featuretable.insert_norm_dlc_arrays import insert_norm_dlc_arrays

query = "SELECT id FROM dlc_table WHERE head_x IS NOT NULL ORDER BY id;"
df = pd.read_sql_query(query, conn)
id_list = df['id'].tolist()

insert_norm_dlc_arrays(id_list, conn, bodypart="head")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Query from database
query = """
SELECT corner1_x, corner1_y,
       corner2_x, corner2_y,
       corner3_x, corner3_y,
       corner4_x, corner4_y
FROM dlc_table
WHERE id = 49;
"""
df = pd.read_sql_query(query, conn)

# Step 2: Compute medians for each corner
corner_pts = []
for i in range(1, 5):
    x_vals = df[f'corner{i}_x'].iloc[0]
    y_vals = df[f'corner{i}_y'].iloc[0]

    if isinstance(x_vals, str):
        x_vals = np.array(eval(x_vals))
    if isinstance(y_vals, str):
        y_vals = np.array(eval(y_vals))

    x_median = np.nanmedian(x_vals)
    y_median = np.nanmedian(y_vals)
    corner_pts.append([x_median, y_median])

corner_pts = np.array(corner_pts)

# Step 3: Plot corners
plt.figure(figsize=(6, 6))
plt.plot(*np.vstack([corner_pts, corner_pts[0]]).T, 'ro-')  # close the loop
for i, (x, y) in enumerate(corner_pts, start=1):
    plt.text(x, y, f"C{i}", fontsize=12, ha='center', va='center')

plt.title("Median Corner Positions for ID 49")
plt.xlabel("x")
plt.ylabel("y")
plt.axis("equal")
plt.grid(True)
plt.show()


## Plot a few trajectories side by side

In [ ]:
query = "SELECT id FROM dlc_table WHERE head_x IS NOT NULL ORDER BY id;"
df = pd.read_sql_query(query, conn)
id_list = df['id'].tolist()
print(id_list)

from Python_scripts.Data_analysis.plot_single_trajectory import plot_single_trajectory
import matplotlib.pyplot as plt

for id in id_list:
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    
    plot_single_trajectory(axs[0], conn, trial_id=id, bodypart_x='head_x', bodypart_y='head_y', 
                           label = f"{id}", title='Original')
    plot_single_trajectory(axs[1], conn, trial_id=id, 
                           label = f"{id}", title='Normalized')
    
    plt.tight_layout()
    plt.show()


## Plot corners of all id

In [ ]:
query = "SELECT id FROM dlc_table WHERE head_x IS NOT NULL ORDER BY id;"
df = pd.read_sql_query(query, conn)
id_list = df['id'].tolist()
print(id_list)

from Python_scripts.Data_analysis.plot_single_trajectory import plot_single_trajectory
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(5, 5))

for id in id_list:
    
    plot_single_trajectory(ax, conn, trial_id=id, bodypart_x='corner1_x', bodypart_y='corner1_y', label=None)
    plot_single_trajectory(ax, conn, trial_id=id, bodypart_x='corner2_x', bodypart_y='corner2_y', label=None)
    plot_single_trajectory(ax, conn, trial_id=id, bodypart_x='corner3_x', bodypart_y='corner3_y', label=None)
    plot_single_trajectory(ax, conn, trial_id=id, bodypart_x='corner4_x', bodypart_y='corner4_y', label=None)

ax.set_title("All Corners Across 10 Trials")
plt.tight_layout()
plt.show()